### Mi primera idea consitió en una sola función para sacar el texto de los tweets, y luego asignarles una puntuación usando el código proveído en las instrucciones del ejercicio que devuelve un diccionario de palabras y su valor de sentimiento. Esta idea funcionó perfectamente, pero me di cuenta que no era muy "pythonic" en que era una función que hizo más de una cosa. También, me di cuenta que cada tweet es un diccionario, entonces para evitar copiar datos, que sería ineficiente, decidí actualizar ese diccionario con el valor del tweet. Al final, opté a crear una clase que se llama "SentimentAnalysis" la cual contiene una función para cada paso, desde crear el diccionario de sentimientos, parse el fichero de tweets para sacar la información de cada tweet, asignar a los tweets la puntuación usando el diccionario, imprimir cada tweet y su puntuación, y al final, por interés de data science, un histograma que muestra la distribución de las puntuaciones de los tweets.

### Esta clase funciona mejor que da más control al usuario.

### Al inciar la clase, se toma dos argumentos, el primero es el path al fichero de Sentimientos, el segunda es el path al fichero de los tweets. Cuando el objeto SentimentAnalysis está creado, se inicia las funciones que crean el diccionario de sentimientos, la lista de diccionarios que cada diccionario es la información sobre un tweet, y que puntuar el sentimiento del texto de cada tweet. Ahora, el usuario puede decidir a simplemente imprimir los tweets y su puntuación, o solo mostrar por la pantalla un histograma y las proporciones de los tweets con un sentimiento neutral, positivo, y negativo. También, el usaurio puede mostrar por pantalla los atributos "sentiments" y "tweets" que muestran los diccionarios creados por su función designada.

### Al final, este ejercicio me hizo analizar en qué tipos de datos estoy manejando. Me parece importante para escribir funciones y ser más eficiente. Esto es más importante en el mundo de Big Data, para que se pueda procesear más rápido. También este ejercicio me dio una oportunidad de aprender y explorar como usar la clase en Python.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
class SentimentAnalysis:
    """
    The SentimentAnalysis object contains sentiments and tweets
    
    Args:
        sentiments_txt: .txt file containing words and their associated sentiment score
        twitter_data: .txt file contain JSON data of tweet records
        
    Attributes:
        sentiments: List of words and their sentiment score
        tweets: List of JSON dictionaries for tweet records
    """

    def __init__(self, sentiments_txt, twitter_data):
        self.set_sentiments(sentiments_txt)
        self.parse_tweets(twitter_data)
        self.score_tweets(self.set_sentiments(sentiments_txt),
                          self.parse_tweets(twitter_data))
        
    def set_sentiments(self, sentiments_txt):
        """
        Generates dictionary of words and their sentiment score
        
        Args:
            sentiments_txt: .txt file containing words and their associated sentiment score
            
        Attributes:
            sentiments: List of words and their sentiment score
        """
        with open(sentiments_txt, 'r') as sentimientos: 
            self.sentiments = {}  # Create empty dictionary
            for linea in sentimientos:
                sentiment, value = linea.split("\t")  # Split word from sentiment score
                self.sentiments[sentiment] = int(value)  # Assign word and sentiment score to sentiments dictionary
    
    def parse_tweets(self, twitter_data):
        """
        Generates list of dictionaries from JSON data of tweet records
        
        Args:
            twitter_data: .txt file contain JSON data of tweet records
        
        Attributes:
            tweets: List of JSON dictionaries for tweet records
        """
        import json
        self.tweets = []  # Create empty list
        with open(twitter_data) as file:
            content = file.read().split('\n')  # Generates list of records
            for line in content:  # Iterate over length of list of records
                try:
                    tweet = json.loads(line)  # Convert each record to JSON format
                    self.tweets.append(tweet)  # Append record to tweets list
                except:
                    continue  # If above fails, continues to next record
    
    def score_tweets(self, tweets, sentiments):
        """
        Scores each tweet using sentiments dictionary
        
        Args:
            tweets: List of tweet record generated by parse_tweets
            sentiments: Dictionary of words and sentiment scores generated by set_sentiments
        """
        for tweet in self.tweets:  # Iterate over each tweet in tweets
            score = 0  # Set score to zero

            if 'text' in tweet.keys():
                for word in tweet['text'].split():  # Iterate over each word in tweet
                    if word.lower() in self.sentiments:  # Returns boolean. True if word is in self.sentiments, False if not.
                        score += self.sentiments.get(word.lower())  # If True is returned, score associated with given word is added to score
                tweet['score'] = score  # Create score key for tweet with corresponding sentiment score
        
    def print_scores(self):
        """
        Prints each tweet and it's associated sentiment score
        """
        for tweet in self.tweets:
            if 'text' in tweet.keys():
                print("EL SIGUENTE TWEET: '" + tweet['text'] + "' TIENE UN SENTIMIENTO ASOCIADO DE: " + str(tweet['score']))
            
    def score_distribution(self):
        """
        Displays a histogram of the distribution of sentiments scores and prints the proportion of tweets with a
        positive, negative, or neutral sentiment
        """
        import matplotlib.pyplot as plt
        import numpy as np
        
        scores = [tweet['score'] for tweet in self.tweets if 'score' in tweet.keys()]  # Generate list of tweet sentiment scores
        plt.hist(scores, bins=range(min(scores), max(scores)), align='left', rwidth=0.9)
        plt.yscale('log')
        plt.xticks(range(min(scores), max(scores)))
        plt.xlabel('Sentiment Score')
        plt.ylabel('Number of Tweets')
        plt.show()
        
        positive = 0  # Set counts to zero
        negative = 0
        neutral = 0

        for score in scores:  # Count scores
            if score == 0:
                neutral += 1
            elif score > 0:
                positive += 1
            elif score < 0:
                negative += 1
        
        num_tweets = len(scores)
        prop_positive = positive / num_tweets * 100  # Calculate proportion of tweets with a positive sentiment
        prop_negative = negative / num_tweets * 100
        prop_neutral = neutral / num_tweets * 100
        
        print(str(prop_positive).split('.')[0] + '% (' + str(positive) + ') of total tweets with a positive sentiment.')
        print(str(prop_negative).split('.')[0] + '% (' + str(negative) + ') of total tweets with a negative sentiment.')
        print(str(prop_neutral).split('.')[0] + '% (' + str(neutral) + ') of total tweets with a neutral sentiment.')

Iniciar la clase SentimentAnalysis pasando como argumentos el path a los ficheros "Sentimientos.txt" y "Tweets.txt"

In [10]:
SA = SentimentAnalysis('/content/drive/MyDrive/Master/Fundamentos de tratamiento de datos para Data Science/Caso Practico/Sentimientos.txt',
                       '/content/drive/MyDrive/Master/Fundamentos de tratamiento de datos para Data Science/Caso Practico/Tweets.txt')

Imprimir por pantalla cada tweet y su sentimiento

In [ ]:
SA.print_scores()

Mostrar por pantalla la histograma de distribución de puntuaciones, también mostrando las proporciones de los tweets con una puntuación neutral, postivo, y negativo

In [ ]:
SA.score_distribution()

In [ ]:
SA.tweets